# @singledispatch

In [1]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br>\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register
def _(n: numbers.Integral) -> str:
    # integer will shown in decimal and hexidecimal forms
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    # float and Decimal will be shown with an approximate fractional equivalent
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

- `bool` is subtype of `numbers.Integral`, but singledispatch is looking first for most specific matching type
- to specify type we can use either type hints or pass the type directly to `@base.register` decorator


In [2]:
print(htmlize({111: 222}))
print(htmlize(print))
print(htmlize('text:\ntext example'))
print(htmlize(12))
print(htmlize([1, 2, 3, 4, 5]))
print(htmlize(True))
print(htmlize(fractions.Fraction(2, 3)))
print(htmlize(1.25))

<pre>{111: 222}</pre>
<pre>&lt;built-in function print&gt;</pre>
<p>text:<br>
text example</p>
<pre>12 (0xc)</pre>
<ul>
<li><pre>1 (0x1)</pre></li>
<li><pre>2 (0x2)</pre></li>
<li><pre>3 (0x3)</pre></li>
<li><pre>4 (0x4)</pre></li>
<li><pre>5 (0x5)</pre></li>
</ul>
<pre>True</pre>
<pre>2/3</pre>
<pre>1.25 (5/4)</pre>
